# Video search
In this example we will be going over the code required to perform video search. This example uses a Vgg model to extract video features that are then used with Milvus to build a system that can perform the searches. 
## Data

This example uses Tumblr's 100 animated gifs as an example to build an end-to-end solution that uses image search video. Readers can use your own video files to build the system.

Download location: https://drive.google.com/file/d/1aYmD74cNfHvYZclFqn2IUTy7cgrMTrXx/view?usp=sharing

## Requirements

| Python Packages | Docker Servers |
| --------------- | -------------- |
| PyMilvus        | Milvus-1.0.0   |
| Redis           | Redis          |

We will assume that you have familiarity with libraries including PyTorch, TorchVision, MatPlotLib, and PIL.

## Up and Running


### 1. Start Milvus Server

```bash
$  docker run -d --name milvus_cpu_1.0.0 --network my-net --ip 10.0.0.2 \
-p 19530:19530 \
-p 19121:19121 \
-v /home/$USER/milvus/db:/var/lib/milvus/db \
-v /home/$USER/milvus/conf:/var/lib/milvus/conf \
-v /home/$USER/milvus/logs:/var/lib/milvus/logs \
-v /home/$USER/milvus/wal:/var/lib/milvus/wal \
milvusdb/milvus:1.0.0-cpu-d030521-1ea92e
```

This demo uses Milvus 1.0. Refer to the [Install Milvus](https://milvus.io/docs/v1.0.0/milvus_docker-cpu.md) for how to install Milvus docker. 

### 2. Start Redis Server

```bash
$ docker run --name some-redis -d redis
```

We are using Redis as a metadata storage service. Code can easily be modified to use python dictionary, but that usually does not work in any use case outside of quick examples.

## Code Overview
### Connecting to Servers

We first start off by connecting to the servers. In this case the docker containers are running on localhost and the ports are the default ports. 

In [ ]:
#Connectings to Milvus and Redis

import redis
import milvus

milv = milvus.Milvus(host = '127.0.0.1', port = 19530)
red = redis.Redis(host = '127.0.0.1', port=6379, db=0)

### Building Collection and Setting Index

The next step involves creating a collection. A collection in Milvus is similar to a table in a relational database, and is used for storing all the vectors. To create a collection, we first must select a name, the dimension of the vectors being stored within, the index_file_size, and metric_type. The index_file_size corresponds to how large each data segmet will be within the collection. More information on this can be found here. The metric_type is the distance formula being used to calculate similarity. In this example we are using the Euclidean distance. 

In [ ]:
#Creating collection

import time

collection_name = "test_collection"
milv.drop_collection(collection_name) 
red.flushdb()
time.sleep(.1)

collection_param = {
            'collection_name': collection_name,
            'dimension': 512,
            'index_file_size': 1024,  # optional
            'metric_type': milvus.MetricType.L2  # optional
            }

status, ok = milv.has_collection(collection_name)

if not ok:
    status = milv.create_collection(collection_param)
    print(status)

After creating the collection we want to assign it an index type. This can be done before or after inserting the data. When done before, indexes will be made as data comes in and fills the data segments. In this example we are using IVF_SQ8 which requires the 'nlist' parameter. Each index types carries its own parameters. More info about this param can be found here.

In [ ]:
#Indexing collection

index_param = {
    'nlist': 512
}

status = milv.create_index(collection_name, milvus.IndexType.IVF_SQ8, index_param)
status, index = milv.get_index_info(collection_name)
print(index)


### Processing and Storing Images

In order to store the images in Milvus, we must first run them through the ResNet model. In this case, we are using the pretrained ResNet-18 model provided by PyTorch. In order to get the feature vectors, we must remove the classifying layer that comes at the end. 

In [ ]:
# Preparing model and dataloader

import torch
from torchvision import datasets, transforms

model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
encoder = torch.nn.Sequential(*(list(model.children())[:-1]))
encoder.eval()

data_dir = "/Users/filiphaltmayer/Documents/datasets/JPEGImages"

In this example we are also using a slightly modified dataloader that also returns the file path of the image. With this dataloader we are also transforming the images into what ResNet model takes as input. 

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    def __getitem__(self, index):
        return super(ImageFolderWithPaths, self).__getitem__(index) + (self.imgs[index][0],)

dataset = ImageFolderWithPaths(data_dir, transform=transforms.Compose([
                                                transforms.Resize(256),
                                                transforms.CenterCrop(224),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]))

dataloader = torch.utils.data.DataLoader(dataset, num_workers=0, batch_size = 256)

Inputting the data involves three major steps. First, the images need to be run through the model. This outputs vectors for each image. Second, these vectors are pushed into Milvus. Milvus then returns the corresponding IDs for the vectors. Third, these IDs and the image filepaths are used as the key and value for storage in Redis. Redis is used so that we can return the original image as a result. 

In [ ]:
# Processing and storing

for inputs, labels, paths in dataloader:
    with torch.no_grad():
        output = encoder(inputs).squeeze()
        output = output.numpy()

    status, ids = milv.insert(collection_name=collection_name, records=output)

    if not status.OK():
        print("Insert failed: {}".format(status))
    else:
        for x in range(len(ids)):
            red.set(str(ids[x]), paths[x])
        print("Added: " + str(len(ids)) + " vectors into: " + collection_name + ".")

In [ ]:
#Helper display function

import matplotlib.pyplot as plt
from PIL import Image

def show_results(query, results, distances):
    
    fig_query, ax_query = plt.subplots(1,1, figsize=(5,5))
    ax_query.imshow(Image.open(query))
    ax_query.axis('off')
    ax_query.set_title("Searched Image")
    
    res_count = len(results)
    fig, ax = plt.subplots(1,res_count,figsize=(5,5))
    for x in range(res_count):
        ax[x].imshow(Image.open(results[x]))
        ax[x].axis('off')
        dist =  str(distances[x])
        dist = dist[0:dist.find('.')+4]
        ax[x].set_title("D: " +dist)
    
        

In [ ]:
# Pulling out random images to search

random_ids = [int(red.randomkey()) for x in range(10)]
search_images = [x.decode("utf-8") for x in red.mget(random_ids)]

### Searching

When searching for an image, we first put the image through the same transformations as the ones used for storing the images. Once transformed, we run the image through the ResNet to get the corresponding vectors. 

In [ ]:
# Processing query images and searching

transform_ops = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
embeddings = [transform_ops(Image.open(x)) for x in search_images]
embeddings = torch.stack(embeddings, dim=0)
    
with torch.no_grad():
    embeddings = encoder(embeddings).squeeze().numpy()

Then we can use these embeddings in a search. The search requires a few arguments. It needs the name of the collection, the vectors being searched for, how many closest vectors to be returned, and the parameters for the index, in this case nprobe. 

In [ ]:
search_sub_param = {
        "nprobe": 16
    }

search_param = {
    'collection_name': collection_name,
    'query_records': embeddings,
    'top_k': 3,
    'params': search_sub_param,
    }

start = time.time()
status, results = milv.search(**search_param)
end = time.time() - start

print("Search took a total of: ", end)

The result of this search contains the IDs and corresponding distances of the top_k closes vectors. We can use the IDs in Redis to get the original image. 

In [ ]:
if status.OK():
    for x in range(len(results)):
        query_file = search_images[x]
        result_files = [red.get(y.id).decode('utf-8') for y in results[x]]
        distances = [y.distance for y in results[x]]
        show_results(query_file, result_files, distances)
else:
    print("Search Failed.")

This is the basic way to do a reverse image search.